# Python Loop Performance
> Andrew Kubera

Q: You have a collection, or some iterator/generator, and you want to apply some function to each item. How do you do it?

If you don't want to create a new collection, just do for loop
```python
for x in things:
    func(x)
```

If you DO want to create a new collection, do NOT
```python
>>> result = []
>>> for x in things:
>>>     result.append(func(x))
```
do
```python
>>> result = [func(x) for x in things]
```

But there *may* be a better way!

`map(function, *iterables)`

```python
result = map(func, things) # WRONG
```

result is now a 'map' object (iterable) - not a collection.

Wrap in a collection (list, tuple, set, etc...) to get your result

```python
result = list(map(func, things))
result == [func(x) for x in things] # True!
```

But which is better? 

In [11]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


I care about performance, so let's do some timeit examples

In [45]:
import sys, platform
print(sys.version)
print("---")
print(*platform.uname()[3:])

3.5.2 (default, Jun 29 2016, 16:37:27) 
[GCC 4.2.1 Compatible Apple LLVM 7.3.0 (clang-703.0.31)]
---
Darwin Kernel Version 15.5.0: Tue Apr 19 18:36:36 PDT 2016; root:xnu-3248.50.21~8/RELEASE_X86_64 x86_64 i386


---
Transform a list of numbers into hex strings

In [53]:
hex(61)

'0x3d'

In [60]:
%timeit [hex(x) for x in range(500)]
%timeit [hex(x) for x in range(5000)]
%timeit [hex(x) for x in range(50000)]

10000 loops, best of 3: 79.5 µs per loop
1000 loops, best of 3: 897 µs per loop
100 loops, best of 3: 10.2 ms per loop


In [61]:
%timeit list(map(hex, range(500)))
%timeit list(map(hex, range(5000)))
%timeit list(map(hex, range(50000)))

10000 loops, best of 3: 70.8 µs per loop
1000 loops, best of 3: 815 µs per loop
100 loops, best of 3: 9.19 ms per loop


In [82]:
"%.3f, %0.3f, %0.3f" % ((79.5 - 70.8) / 0.795, (897 - 815) / 8.97, (10.2 - 9.19) / .102)

'10.943, 9.142, 9.902'

There should be no reason why tuples or lists should be more efficient

In [65]:
%timeit tuple(map(hex, range(500)))
%timeit tuple(map(hex, range(5000)))
%timeit tuple(map(hex, range(50000)))

10000 loops, best of 3: 70.2 µs per loop
1000 loops, best of 3: 852 µs per loop
100 loops, best of 3: 9.39 ms per loop


Let's try again with a `lambda` calling hex instead of just hex.

There is no reason why this should be faster, and I don't expect it to be

In [67]:
%timeit list(map(lambda i: hex(i), range(500)))
%timeit list(map(lambda i: hex(i), range(5000)))
%timeit list(map(lambda i: hex(i), range(50000)))

10000 loops, best of 3: 130 µs per loop
1000 loops, best of 3: 1.43 ms per loop
100 loops, best of 3: 15.8 ms per loop


Hmm, looks like map is winning if the method is right there, but using the lambda slows it down by A LOT.

Give a little math to the library - hex the square of each number

In [85]:
%timeit -n 1000 [hex(x ** 2) for x in range(500)]
%timeit -n 500  [hex(x ** 2) for x in range(5000)]
%timeit -n 200  [hex(x ** 2) for x in range(50000)]

1000 loops, best of 3: 274 µs per loop
500 loops, best of 3: 2.89 ms per loop
200 loops, best of 3: 32.7 ms per loop


In [84]:
%timeit -n 1000 list(map(lambda i: hex(i ** 2), range(500)))
%timeit -n 500  list(map(lambda i: hex(i ** 2), range(5000)))
%timeit -n 200  list(map(lambda i: hex(i ** 2), range(50000)))

1000 loops, best of 3: 346 µs per loop
500 loops, best of 3: 3.5 ms per loop
200 loops, best of 3: 37.5 ms per loop


In [86]:
"%.3f, %0.3f, %0.3f" % ((274 - 346) / 2.74, (2.89 - 3.50) / .0289, (32.7 - 37.5) / 0.327)

'-26.277, -21.107, -14.679'

hmm, decrease in efficiency by ~20%!!

In [76]:
%timeit -n 1000 list((hex(i ** 2) for i in range(500)))
%timeit -n 1000 list((hex(i ** 2) for i in range(5000)))
%timeit -n 1000 list((hex(i ** 2) for i in range(50000)))

1000 loops, best of 3: 307 µs per loop
1000 loops, best of 3: 3.18 ms per loop
1000 loops, best of 3: 34.6 ms per loop


Huh, that's actually worse than the list comprehension from before, too bad.

In [77]:
%timeit -n 1000 list(map(hex, (i ** 2 for i in range(500))))
%timeit -n 400  list(map(hex, (i ** 2 for i in range(5000))))
%timeit -n 100  list(map(hex, (i ** 2 for i in range(50000))))

1000 loops, best of 3: 304 µs per loop
400 loops, best of 3: 3.17 ms per loop
100 loops, best of 3: 34.7 ms per loop


In [78]:
%timeit -n 1000 [hex(i) for i in (i ** 2 for i in range(500))]
%timeit -n 400  [hex(i) for i in (i ** 2 for i in range(5000))]
%timeit -n 100  [hex(i) for i in (i ** 2 for i in range(50000))]

1000 loops, best of 3: 324 µs per loop
400 loops, best of 3: 3.31 ms per loop
100 loops, best of 3: 35.5 ms per loop


In [79]:
def hexsquare(x):
    return hex(x ** 2)

%timeit -n 1000 list(map(hexsquare, range(500)))
%timeit -n 500  list(map(hexsquare, range(5000)))
%timeit -n 100  list(map(hexsquare, range(50000)))

1000 loops, best of 3: 358 µs per loop
500 loops, best of 3: 3.46 ms per loop
100 loops, best of 3: 38 ms per loop


The lambda performs better than the function by a marginal amount ~2%